In [13]:
%matplotlib inline

/home/mike/ve/ml/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/mike/ve/ml/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [14]:
from __future__ import division, print_function
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [10]:
#List out our bandits. Currently bandit 4 
#(index 3) is set to most often provide a positive reward.

class BanditBot:
    def __init__(self, p=0.):
        self.p = p
        
    def __call__(self, *args, **kwargs):
        result = np.random.randn(1)
        if result > self.p:
            return 1 #return a positive reward.
        else:
            return -1 #return a negative reward.

bandits = [0.2,0,-0.2,-5]
num_bandits = len(bandits)
def pullBandit(bandit):
    #Get a random number.
    result = np.random.randn(1)
    if result > bandit:
        #return a positive reward.
        return 1
    else:
        #return a negative reward.
        return -1

In [3]:
#init our agent

tf.reset_default_graph()

#These two lines established the feed-forward part of the network. 
#This does the actual choosing.
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

#The next six lines establish the training proceedure. 
#We feed the reward and chosen action into the network
#to compute the loss, and use it to update the network.
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [11]:
bots = [BanditBot(p) for p in bandits]

In [12]:
total_episodes = 1000 #Set total number of episodes to train agent on.
total_reward = np.zeros(num_bandits) #Set scoreboard for bandits to 0.
e = 0.1 #Set the chance of taking a random action.

init = tf.initialize_all_variables()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        #Choose either a random action or one from our network.
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        
#         reward = pullBandit(bandits[action]) #Get our reward from picking one of the bandits.
        reward = bots[action]() #Get our reward from picking one of the bandits.

        
        #Update the network.
        _,resp,ww = sess.run([update,responsible_weight,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #Update our running tally of scores.
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the {} bandits {}".format(str(num_bandits), str(total_reward)))
        i+=1
print("The agent thinks bandit {} is the most promising....".format(str(np.argmax(ww)+1)))
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Running reward for the 4 bandits [-1.  0.  0.  0.]
Running reward for the 4 bandits [  0.  -2.  -1.  38.]
Running reward for the 4 bandits [  0.  -2.   0.  87.]
Running reward for the 4 bandits [   0.   -2.   -1.  134.]
Running reward for the 4 bandits [   0.    0.    0.  181.]
Running reward for the 4 bandits [  -2.    0.    0.  229.]
Running reward for the 4 bandits [  -3.    0.    0.  278.]
Running reward for the 4 bandits [  -4.    0.    1.  326.]
Running reward for the 4 bandits [  -4.    0.    1.  376.]
Running reward for the 4 bandits [  -3.    0.    2.  424.]
Running reward for the 4 bandits [  -1.   -1.    4.  469.]
Running reward for the 4 bandits [  -2.    1.    3.  513.]
Running reward for the 4 bandits [  -1.    1.    5.  558.]
Running reward for the 4 bandits [   0.    3.    4.  604.]
Running reward for the 4 bandits [   1.    3.    4.  653.]
Running reward for the 4 bandits [   1.    2.    4.  702.